# GeoAlchemy2 ORM Test -- Import ODHF

Source: [https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html](https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html)

In [ ]:
# read configuration
import psycopg2, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

#conn_string = "host=%(SERVER)s user=%(USERNAME)s dbname=%(DATABASE)s password=%(PASSWORD)s sslmode=%(SSLMODE)s" % config
#conn = psycopg2.connect(conn_string) 
#print("Connection established")


# Some psycopg2 intialization

from register_adapters import register_adapters

register_adapters()

## Connect to the DB

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://%(USERNAME)s:%(PASSWORD)s@%(SERVER)s/%(DATABASE)s?sslmode=%(SSLMODE)s" % config,
    client_encoding="utf8", 
    echo=True
)

## Declare a Mapping

We will try to map the CSD digital boundary files 

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.path.abspath(""), "models"))

from models import HealthFacility

## Create the Table in the Database

In [ ]:
try:
    HealthFacility.__table__.drop(engine)
except:
    pass

HealthFacility.__table__.create(engine)

In [ ]:
import pandas as pd
import os, zipfile

zf = zipfile.ZipFile(os.path.join(os.path.abspath(""), "data", "ODHF_v1.1.zip")) 
df = pd.read_csv(zf.open('ODHF_v1.1/odhf_v1.1.csv'), encoding="cp1252",keep_default_na=False)

"""
df.Full_Addr = df.Full_Addr.astype(str)
df.Unit = df.Unit.astype(str)
df.Street_No = df.Street_No.astype(str)
df.Street_Name = df.Street_Name.astype(str)
df.City = df.City.astype(str)
df.Prov_Terr = df.Prov_Terr.astype(str)
df.Postal_Code = df.Postal_Code.astype(str)
"""

print(df.info())

df = df[(df.latitude != "..") & (df.longitude != "..") & (df.latitude != "") & (df.longitude != "")]

df

In [ ]:
df.odhf_facility_type.unique()

## Create a Session

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()


In [ ]:
from uuid import uuid4

# helper function to create table entries from GeoDataFrame
def insert_health(session, x):

    row = HealthFacility( 
                index = uuid4(),
                facility_name = x.facility_name.strip(),
                facility_type = x.odhf_facility_type.lower().strip(),
                full_addr = x.source_format_str_address.strip(),   
                unit = x.unit.strip(),      
                street_no = x.street_no.strip(),   
                street_name = x.street_name.strip(),
                city = x.city.strip(),           
                province = x.province.strip(),    
                postal_code = x.postal_code.strip(),
                geometry = 'POINT(%s %s)' % (x.longitude, x.latitude)
                )


    session.add(row)


session.rollback()

df.apply(lambda x: insert_health(session, x), axis=1)

# commit changes
session.commit()

## Query

In [ ]:
# Get all health facilities
hf_list = session.query(HealthFacility).order_by(HealthFacility.facility_name).all()

for i in hf_list[:10]:
    print(i.facility_name, i.facility_type, i.full_addr, i.CSD.CSDUID, i.CSD.name)
